Important library

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

Load validation and training dataset

dataset available at : https://www.kaggle.com/datasets/msambare/fer2013

In [ ]:
# Step 1: Load and preprocess the FER2013 dataset or any we want to use
# Assume dataset is in 'data/train' and 'data/validation' in folders
# dataset available at : https://www.kaggle.com/datasets/msambare/fer2013
batch_size = 32
img_size = (224, 224)

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)


val_datagen = ImageDataGenerator(rescale=1.0/255.0)


train_data = train_datagen.flow_from_directory(
    'data/train',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_data = val_datagen.flow_from_directory(
    'data/validation',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)



model training

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))# Step 2: Load the pretrained ResNet50 model

for layer in base_model.layers:# Freeze the base model layers
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
output = Dense(train_data.num_classes, activation='softmax')(x) # Add custom layers


model = Model(inputs=base_model.input, outputs=output) # Combine base and custom layers

model.compile(optimizer=Adam(learning_rate=0.001),        #Compile the model
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Train the model

In [ ]:

epochs = 10
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=epochs
)


for layer in base_model.layers[-20:]:
    layer.trainable = True


model.compile(optimizer=Adam(learning_rate=1e-5),   # Recompile with a lower learning rate
              loss='categorical_crossentropy',
              metrics=['accuracy'])

fine_tune_epochs = 5
fine_tune = model.fit(
    train_data,
    validation_data=val_data,
    epochs=fine_tune_epochs
)                              # Fine-tune the model



comparision variouse matrices

In [ ]:
import matplotlib.pyplot as plt

# Fine-tune the model
fine_tune_epochs = 5
fine_tune = model.fit(
    train_data,
    validation_data=val_data,
    epochs=fine_tune_epochs
)

# Save the model
model.save('emotion_recognition_model.h5')

# Print final accuracy on validation set
val_accuracy = fine_tune.history['val_accuracy'][-1]
print(f"Final Validation Accuracy: {val_accuracy * 100:.2f}%")

# Plotting training and validation accuracy
plt.figure(figsize=(12, 6))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(fine_tune.history['accuracy'], label='Training Accuracy')
plt.plot(fine_tune.history['val_accuracy'], label='Validation Accuracy')
plt.title('Epoch vs. Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(fine_tune.history['loss'], label='Training Loss')
plt.plot(fine_tune.history['val_loss'], label='Validation Loss')
plt.title('Epoch vs. Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()